In [2]:
import pandas as pd
from datetime import datetime
import requests
import numpy as np
from polygon.rest import RESTClient
from tqdm import tqdm
import warnings
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import json

api_key = 'hFrBS7nzcaLTa8mplO1ejm44DI4EscDM'
client = RESTClient(api_key)

warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=pd_errors.SettingWithCopyWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [3]:
with open('dlist.json', 'r') as file:
    dlist = json.load(file)
df = pd.read_csv('save2.csv')
df['ticker'] = df['key'].str.split('_').str[0]
df['date'] = df['key'].str.split('_').str[1]
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,key,%c1m,%c1m_lag1,%c1m_lag2,%c1m_lag3,%c2m,%c2m_lag1,...,%c270m_screen,%c285m_screen,%c300m_screen,%c315m_screen,%c330m_screen,%c345m_screen,%c360m_screen,%c375m_screen,ticker,date
0,0,0,991,AA_2021-03-18,0.001304,-0.001046,-0.000497,0.001117,0.000257,0.000620,...,0,0,0,0,0,0,0,0,AA,2021-03-18
1,1,1,1170,AA_2021-12-01,0.001808,-0.000352,-0.002130,-0.001119,0.001455,-0.003247,...,0,0,0,0,1,0,0,0,AA,2021-12-01
2,2,2,1372,AA_2022-09-21,-0.000755,-0.000833,-0.000215,-0.000086,-0.001588,-0.000301,...,0,0,0,0,0,0,0,0,AA,2022-09-21
3,3,3,1402,AA_2022-11-02,-0.000124,-0.000995,0.000483,0.000777,-0.001119,0.001260,...,0,0,0,0,0,0,0,0,AA,2022-11-02
4,4,4,1686,AA_2023-12-20,-0.000746,-0.000092,-0.000807,-0.000470,-0.000837,-0.001277,...,0,0,0,0,0,0,0,0,AA,2023-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232661,232661,232661,212630,ZYXI_2023-05-04,-0.000972,0.002065,0.003576,0.001226,0.001092,0.004806,...,0,1,1,1,1,1,1,1,ZYXI,2023-05-04
232662,232662,232662,212690,ZYXI_2023-08-01,0.000128,0.000116,0.001977,0.000652,0.000244,0.002629,...,0,0,1,1,1,1,1,0,ZYXI,2023-08-01
232663,232663,232663,212746,ZYXI_2023-10-19,0.000065,0.002243,0.002117,0.002227,0.002309,0.004349,...,0,0,0,0,0,0,0,0,ZYXI,2023-10-19
232664,232664,232664,212765,ZYXI_2023-11-15,0.000045,-0.000079,0.000284,0.000397,-0.000034,0.000681,...,0,0,0,0,0,0,0,0,ZYXI,2023-11-15


<h1>Get daily aggs</h1>

In [4]:
def unix(date_str,hour,minute):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    #Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    #Create datetime object
    est_time = datetime(date.year, date.month, date.day, hour, minute, 0)
    #Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in milliseconds
    unix_timestamp_ms = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1000)
    return unix_timestamp_ms

In [5]:
def daily_aggs(ticker,date):
    try:
        #setup
        ret_dict = {}
        s_date = dlist[dlist.index(date) - 3]
        e_date = dlist[dlist.index(date) + 1]
        #call API
        data = client.get_aggs(ticker,
                                multiplier=1,
                                timespan='day',
                                adjusted='false',
                                from_=s_date,
                                to=e_date)
        df = pd.DataFrame(data)
        #Next open data
        odf = df[df['timestamp'] == unix(e_date,0,0)]
        if not odf.empty:
            ret_dict['o1'] = odf['open'].values[0]
        else:
            ret_dict['o1'] = np.nan
        #Trade day data
        tdf = df[df['timestamp'] == unix(date,0,0)]
        if not tdf.empty:
            ret_dict['o0'] = tdf['open'].values[0]
            ret_dict['c0'] = tdf['close'].values[0]
        else:
            ret_dict['o0'] = ret_dict['c0'] = np.nan
        #previous days
        for x in range(1,4):
            pdf = df[df['timestamp'] == unix(dlist[dlist.index(date) - x],0,0)]
            if not pdf.empty:
                ret_dict[f'c-{x}'] = pdf['close'].values[0]
            else:
                ret_dict[f'c-{x}'] = np.nan
        return ret_dict
    except Exception as e:
        print(ticker,date,e)
        ret_dict['o1'] = ret_dict['o0'] = ret_dict['c0'] = ret_dict['c-1'] = ret_dict['c-2'] = ret_dict['c-3'] = np.nan
        return ret_dict
daily_aggs('AAPL','2024-01-05')

{'o1': 182.085,
 'o0': 181.99,
 'c0': 181.18,
 'c-1': 181.91,
 'c-2': 184.25,
 'c-3': 185.64}

In [6]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(daily_aggs, df['ticker'], df['date']), total=len(df)))
daily_df = pd.DataFrame(results)
df = pd.concat([df, daily_df], axis=1)
df.info()

100%|██████████| 232666/232666 [45:36<00:00, 85.02it/s]  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232666 entries, 0 to 232665
Columns: 166 entries, Unnamed: 0.2 to c-3
dtypes: float64(127), int64(36), object(3)
memory usage: 294.7+ MB


<h1>Get next day open data</h1>

In [7]:
def unix_s(date_str, hour, minute, seconds):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    # Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    # Create datetime object
    est_time = datetime(date.year, date.month, date.day, hour, minute, seconds)
    # Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in milliseconds
    unix_timestamp_ms = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1000)
    return unix_timestamp_ms
#unix_s('2024-01-01',9,30,1)

In [8]:
def est_seconds(unix_ms_timestamp):
    # Convert Unix timestamp in milliseconds to seconds
    unix_seconds = unix_ms_timestamp / 1000.0
    # Create a datetime object from the Unix timestamp
    utc_time = datetime.utcfromtimestamp(unix_seconds)
    # Define the UTC and EST timezones
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    # Localize the UTC datetime object to UTC timezone
    utc_time = utc_zone.localize(utc_time)
    # Convert the UTC time to EST
    est_time = utc_time.astimezone(est_zone)
    est_time = est_time.replace(tzinfo=None)
    # Create a reference time for 9:30 AM EST on the same date
    reference_time = datetime(est_time.year, est_time.month, est_time.day, 9, 30)
    # Calculate the difference in seconds between the given time and 9:30 AM
    time_diff = est_time - reference_time
    # Return the total seconds
    return int(time_diff.total_seconds())
#est_seconds(1704119401000)

In [9]:
def next_open_aggs(ticker,date):
    try:
        #setup
        ret_dict = {}
        next_date = dlist[dlist.index(date) + 1]
        #call API
        data = client.get_aggs(ticker,
                                multiplier=1,
                                timespan='second',
                                adjusted='false',
                                from_=unix_s(next_date,9,30,1),
                                to=unix_s(next_date,15,59,0),
                                limit = 500)
        df = pd.DataFrame(data)
        if df.empty:
            ret_dict['first_tick'] = ret_dict['first_tick_time'] = ret_dict['9:30_vw'] = np.nan
            ret_dict['9:30_dv'] = 0
            return ret_dict
        df['dv'] = df['volume'] * df['vwap']
        #first tick
        ret_dict['first_tick'] = df['open'].iloc[0]
        ret_dict['first_tick_time'] = est_seconds(df['timestamp'].iloc[0])
        #9:30 after open
        df1 = df[df['timestamp'] < unix_s(next_date,9,31,0)]
        if not df1.empty:
            ret_dict['9:30_dv'] = (df1['vwap'] * df1['volume']).sum()
            ret_dict['9:30_vw'] = round((ret_dict['9:30_dv']/df1['volume'].sum()),6)
        else:
            ret_dict['9:30_dv'] = 0
            ret_dict['9:30_vw'] = np.nan
        return ret_dict
    except Exception as e:
        print(ticker,date,e)
        ret_dict['first_tick'] = ret_dict['first_tick_time'] = ret_dict['9:30_vw'] = np.nan
        ret_dict['9:30_dv'] = 0
        return ret_dict
next_open_aggs('A','2024-01-05')

{'first_tick': 130.365, 'first_tick_time': 76, '9:30_dv': 0, '9:30_vw': nan}

In [10]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(next_open_aggs, df['ticker'], df['date']), total=len(df)))
minute_df = pd.DataFrame(results)
df = pd.concat([df, minute_df], axis=1)
df.info()

100%|██████████| 232666/232666 [1:07:54<00:00, 57.10it/s] 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232666 entries, 0 to 232665
Columns: 170 entries, Unnamed: 0.2 to 9:30_vw
dtypes: float64(131), int64(36), object(3)
memory usage: 301.8+ MB


In [11]:
df.to_csv('save3.csv')

<h1>Get bid/ask data</h1>

In [3]:
df = pd.read_csv('save3.csv')

In [4]:
def est_mins(unix_ns_timestamp):
    # Convert Unix timestamp in nanoseconds to seconds
    unix_seconds = unix_ns_timestamp / 1_000_000_000.0
    # Create a datetime object from the Unix timestamp
    utc_time = datetime.utcfromtimestamp(unix_seconds)
    # Define the UTC and EST timezones
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    # Localize the UTC datetime object to UTC timezone
    utc_time = utc_zone.localize(utc_time)
    # Convert the UTC time to EST
    est_time = utc_time.astimezone(est_zone)
    est_time = est_time.replace(tzinfo=None)
    return est_time.strftime('%H:%M')

In [5]:
def unixn(date_str, hour, minute, second):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    # Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    # Create datetime object with seconds
    est_time = datetime(date.year, date.month, date.day, hour, minute, second)
    # Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in nanoseconds
    unix_timestamp_ns = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1e9)
    return unix_timestamp_ns

In [6]:
def nbbo_close(ticker,date_string,key):
    try:
        ndict = {}
        ndict['key'] = key
        lb = unixn(date_string,9,30,00)
        ub = unixn(date_string,15,59,59)
        response = requests.get(f'https://api.polygon.io/v3/quotes/{ticker}?timestamp.gte={lb}&timestamp.lt={ub}&order=desc&sort=timestamp&limit=1&apiKey={api_key}')
        data = response.json()
        df = pd.DataFrame(data['results'])
        if df.empty:
            return ndict
        ndict['close_bid'] = df['bid_price'].values[0]
        ndict['close_mid'] = (df['bid_price'].values[0] + df['ask_price'].values[0]) / 2
        ndict['close_ask'] = df['ask_price'].values[0]
        ndict['close_ask_depth'] = round(df['ask_price'].values[0] * df['ask_size'].values[0] * 100, 0)
        ndict['close_quote_time'] = est_mins(df['sip_timestamp'].values[0])
        return ndict
    except Exception as e:
        print(f'{ticker,date_string}: {e}')
        return ndict
nbbo_close('STAB', '2021-10-14','k')

{'key': 'k',
 'close_bid': 2.8,
 'close_mid': 2.815,
 'close_ask': 2.83,
 'close_ask_depth': 849.0,
 'close_quote_time': '15:59'}

In [7]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(nbbo_close, df['ticker'], df['date'],df['key']), total=len(df['ticker'])))
closedf = pd.DataFrame(results)
df = df.merge(closedf, on='key', how='right')
closedf.info()

 44%|████▍     | 102662/232666 [56:30<49:06, 44.12it/s]  

('IMRX', '2024-05-17'): HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v3/quotes/IMRX?timestamp.gte=1715952600000000000&timestamp.lt=1715975999000000000&order=desc&sort=timestamp&limit=1&apiKey=hFrBS7nzcaLTa8mplO1ejm44DI4EscDM (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14e148110>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


100%|██████████| 232666/232666 [1:54:10<00:00, 33.96it/s]   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232666 entries, 0 to 232665
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   key               232666 non-null  object 
 1   close_bid         232550 non-null  float64
 2   close_mid         232550 non-null  float64
 3   close_ask         232550 non-null  float64
 4   close_ask_depth   232550 non-null  float64
 5   close_quote_time  232550 non-null  object 
dtypes: float64(4), object(2)
memory usage: 10.7+ MB


In [8]:
df.to_csv('save4.csv')

In [9]:
with open('dlist.json', 'r') as file:
    dlist = json.load(file)

In [10]:
def nbbo_open(ticker,date_string,key):
    try:
        ndict = {}
        ndict['key'] = key
        date_stringp1 = dlist[dlist.index(date_string) + 1]
        lb = unixn(date_stringp1,9,30,1)
        ub = unixn(date_stringp1,16,00,0)
        response = requests.get(f'https://api.polygon.io/v3/quotes/{ticker}?timestamp.gte={lb}&timestamp.lt={ub}&order=asc&sort=timestamp&limit=1&apiKey={api_key}')
        data = response.json()
        df = pd.DataFrame(data['results'])
        if not df.empty:
            ndict['open_bid'] = df['bid_price'].values[0]
            ndict['open_mid'] = (df['bid_price'].values[0] + df['ask_price'].values[0]) / 2
            ndict['open_ask'] = df['ask_price'].values[0]
            ndict['open_bid_depth'] = round(df['bid_price'].values[0] * df['bid_size'].values[0] * 100, 0)
            ndict['open_quote_time'] = est_mins(df['sip_timestamp'].values[0])
        else:
            return ndict
        return ndict
    except Exception as e:
        print(e)
        return ndict
nbbo_open('VS','2022-02-25','balls')

{'key': 'balls',
 'open_bid': 1.23,
 'open_mid': 1.23,
 'open_ask': 1.23,
 'open_bid_depth': 123.0,
 'open_quote_time': '09:30'}

In [11]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(nbbo_open, df['ticker'], df['date'], df['key']), total=len(df['ticker'])))
opendf = pd.DataFrame(results)
df = df.merge(opendf, on='key', how='right')
opendf.info()

100%|██████████| 232666/232666 [1:15:11<00:00, 51.57it/s]  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232666 entries, 0 to 232665
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   key              232666 non-null  object 
 1   open_bid         232468 non-null  float64
 2   open_mid         232468 non-null  float64
 3   open_ask         232468 non-null  float64
 4   open_bid_depth   232468 non-null  float64
 5   open_quote_time  232468 non-null  object 
dtypes: float64(4), object(2)
memory usage: 10.7+ MB


In [12]:
df.to_csv('save5.csv')

<h1>Adjust data for splits</h1>

In [4]:
df = pd.read_csv('save5.csv')

In [5]:
def splits(ticker):
    try:
        response = requests.get(f'https://api.polygon.io/v3/reference/splits?ticker={ticker}&limit=100&apiKey=hFrBS7nzcaLTa8mplO1ejm44DI4EscDM')
        data = response.json()
        sdf = pd.DataFrame(data['results'])
        if sdf.empty:
            return {}
        sdf['raw_factor'] = sdf['split_from'] / sdf['split_to']
        records_list = sdf.to_dict(orient='records')
        single_dict = records_list if records_list else {}
        return single_dict
    except Exception as e:
        print(f"An exception error occurred for {ticker} on {date}: {e}")
        return {}
splits('EZGO')

[{'execution_date': '2024-04-12',
  'id': 'E24344ef530d56e1636c10e0a467dd99e758250cbbe9d14f465ac4334ab7081b7',
  'split_from': 40,
  'split_to': 1,
  'ticker': 'EZGO',
  'raw_factor': 40.0}]

In [6]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(splits, df['ticker'].unique()), total=len(df['ticker'].unique())))
results = [item for sublist in results for item in sublist]
split_df = pd.DataFrame(results)
split_df.info()

100%|██████████| 5997/5997 [01:51<00:00, 53.99it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3445 entries, 0 to 3444
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   execution_date  3445 non-null   object 
 1   id              3445 non-null   object 
 2   split_from      3445 non-null   float64
 3   split_to        3445 non-null   float64
 4   ticker          3445 non-null   object 
 5   raw_factor      3445 non-null   float64
dtypes: float64(3), object(3)
memory usage: 161.6+ KB


In [7]:
ind_col_fac = []
for ticker, split_date, factor in tqdm(zip(split_df['ticker'],split_df['execution_date'],split_df['raw_factor']),total=len(split_df)):
    tdf = df[df['ticker'] == ticker]
    for _,row in tdf.iterrows():
        date_range = dlist[dlist.index(row['date']) - 3: dlist.index(row['date']) + 2]
        if split_date in date_range:
            pos = date_range.index(split_date)
            if pos == 1:
                ind_col_fac.append([row.name,['c-3'],factor])
            elif pos == 2:
                ind_col_fac.append([row.name,['c-3','c-2'],factor])
            elif pos == 3:
                ind_col_fac.append([row.name,['c-3','c-2','c-1'],factor])
            elif pos == 4:
                ind_col_fac.append([row.name,['c-3','c-2','c-1','o0','c0','15:59_o','15:58_c','15:59_vw','close_bid','close_mid','close_ask'],factor])

  0%|          | 0/3445 [00:00<?, ?it/s]

100%|██████████| 3445/3445 [01:01<00:00, 55.76it/s]


In [8]:
for ind,cols,fac in tqdm(ind_col_fac,total=len(ind_col_fac)):
    for col in cols:
        df.at[ind, col] = df.at[ind, col] * fac

100%|██████████| 1624/1624 [00:00<00:00, 3874.60it/s]


In [9]:
df.to_csv('save6.csv')

<h1>Create features</h1>

In [10]:
#previous moves
df['%cintra'] = round((df['15:58_c']/df['o0'])-1,6)
df['%c1d'] = round((df['15:58_c']/df['c-1'])-1,6)
df['%cpah'] = round((df['o0']/df['c-1'])-1,6)
df['%c2d'] = round((df['15:58_c']/df['c-2'])-1,6)
df['%c3d'] = round((df['15:58_c']/df['c-3'])-1,6)

In [11]:
#returns
df['vwap_return'] = round((df['9:30_vw']/df['15:59_vw'])-1,6)
df['vwap_ft_return'] = round((df['first_tick']/df['15:59_vw'])-1,6)
df['ask_bid_return'] = round((df['open_bid']/df['close_ask'])-1,6)
df['mid_return'] = round((df['open_mid']/df['close_mid'])-1,6)
df['spread_entry'] = np.where(df['15:59_vw'].notna(), df['15:59_vw'], df['close_ask'])
df['spread_exit'] = np.where(df['9:30_vw'].notna(), df['9:30_vw'], df['open_bid'])
df['vwap_sfill_return'] = round((df['spread_exit']/df['spread_entry'])-1,6)

In [12]:
df.to_csv('dirty_main.csv')